In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

ModuleNotFoundError: No module named 'catboost'

In [2]:
train_dir = "C:/Users/MBR/Desktop/task cv/Alzheimer_s Dataset/Alzheimer_s Dataset/train"
test_dir = "C:/Users/MBR/Desktop/task cv/Alzheimer_s Dataset/Alzheimer_s Dataset/test"

In [3]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [4]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [5]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [6]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [7]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

c:\Users\SHAHD BADR\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

  2/161 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step  

c:\Users\SHAHD BADR\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step


In [9]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step


In [10]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3270886	total: 158ms	remaining: 2m 37s
1:	learn: 1.2773483	total: 178ms	remaining: 1m 29s
2:	learn: 1.2380031	total: 199ms	remaining: 1m 6s
3:	learn: 1.2021284	total: 222ms	remaining: 55.3s
4:	learn: 1.1726747	total: 245ms	remaining: 48.8s
5:	learn: 1.1472653	total: 266ms	remaining: 44.1s
6:	learn: 1.1239137	total: 288ms	remaining: 40.9s
7:	learn: 1.1026559	total: 312ms	remaining: 38.7s
8:	learn: 1.0834344	total: 334ms	remaining: 36.8s
9:	learn: 1.0657473	total: 357ms	remaining: 35.4s
10:	learn: 1.0490749	total: 381ms	remaining: 34.3s
11:	learn: 1.0351133	total: 407ms	remaining: 33.5s
12:	learn: 1.0231106	total: 432ms	remaining: 32.8s
13:	learn: 1.0107956	total: 454ms	remaining: 32s
14:	learn: 1.0014043	total: 476ms	remaining: 31.2s
15:	learn: 0.9916122	total: 499ms	remaining: 30.7s
16:	learn: 0.9819277	total: 524ms	remaining: 30.3s
17:	learn: 0.9736383	total: 550ms	remaining: 30s
18:	learn: 0.9646252	total: 572ms	remaining: 29.5s
19:	learn: 0.9566523	total: 594ms	remaining

In [11]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [12]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 56.22%


In [13]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [14]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [16]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model("D:/ai-ml/computer vision/Task4/2/cnn_feature_extractor6.h5")
catboost_classifier = joblib.load("D:/ai-ml/computer vision/Task4/2/catboost_classifier.pkl")  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = "D:/ai-ml/computer vision/Task4/Alzheimer_s Dataset/Alzheimer_s Dataset/Alzheimer_s Dataset/test/NonDemented/26 (62).jpg"
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
The image is classified as: NonDemented


C:\Users\SHAHD BADR\AppData\Local\Temp\ipykernel_19060\529946340.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
